In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

### Tokenizer

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [4]:
from pyspark.sql.functions import col, udf

In [5]:
from pyspark.sql.types import IntegerType

In [24]:
sentence_df = spark.createDataFrame([
    (0, 'Hi I like Spark'),
    (1, 'Too bad java cant use these classes'),
    (2, 'Logistic,regression,models,are,neat'),
], ['id', 'sentence'])

In [7]:
sentence_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|     Hi I like Spark|
|  1|Too bad java cant...|
|  2|Logistic,regressi...|
+---+--------------------+



In [8]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [10]:
regexTokenizer = RegexTokenizer(
    inputCol='sentence',
    outputCol='words',
    pattern='\\W')

In [11]:
# user defined function
count_tokens = udf(lambda words: len(words), IntegerType())

In [12]:
tokenized = tokenizer.transform(sentence_df)

In [13]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|     Hi I like Spark|[hi, i, like, spark]|
|  1|Too bad java cant...|[too, bad, java, ...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [14]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|     Hi I like Spark|[hi, i, like, spark]|     4|
|  1|Too bad java cant...|[too, bad, java, ...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [15]:
# default tokenizer didn't work for the last sentence
# it didn't know to split on commas
# we can fix with regex

In [16]:
rg_tokenized = regexTokenizer.transform(sentence_df)

In [17]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|     Hi I like Spark|[hi, i, like, spark]|     4|
|  1|Too bad java cant...|[too, bad, java, ...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



### Removing Stop Words

In [18]:
from pyspark.ml.feature import StopWordsRemover

In [22]:
sentence_df_2 = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])
    ], ['id', 'tokens'])

In [20]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filterd')

In [21]:
remover.transform(sentence_df_2).show()

+---+--------------------+--------------------+
| id|              tokens|             filterd|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



### N-Grams

In [23]:
from pyspark.ml.feature import NGram

In [31]:
sentence_df_3 = spark.createDataFrame([
    (0, ['Hi', 'I', 'like', 'Spark']),
    (1, ['Too', 'bad', 'java', 'cant', 'use', 'these', 'classes']),
    (2, ['Logistic', 'regression', 'models', 'are', 'neat']),
], ['id', 'words'])

In [32]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [33]:
ngram.transform(sentence_df_3).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I like, like Spark]                                        |
|[Too bad, bad java, java cant, cant use, use these, these classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



### TF-IDF

In [35]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [36]:
sentence_df = spark.createDataFrame([
    (0, 'Hi I like Spark'),
    (1, 'Too bad java cant use these classes'),
    (2, 'Logistic,regression,models,are,neat'),
], ['id', 'sentence'])

In [37]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [38]:
words_data = tokenizer.transform(sentence_df)

In [41]:
words_data.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|     Hi I like Spark|[hi, i, like, spark]|
|  1|Too bad java cant...|[too, bad, java, ...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [42]:
hasing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [43]:
featurized_data = hasing_tf.transform(words_data)

In [44]:
featurized_data.show()

+---+--------------------+--------------------+--------------------+
| id|            sentence|               words|         rawFeatures|
+---+--------------------+--------------------+--------------------+
|  0|     Hi I like Spark|[hi, i, like, spark]|(262144,[24417,49...|
|  1|Too bad java cant...|[too, bad, java, ...|(262144,[55551,88...|
|  2|Logistic,regressi...|[logistic,regress...|(262144,[84234],[...|
+---+--------------------+--------------------+--------------------+



In [45]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [46]:
idf_model = idf.fit(featurized_data)

In [47]:
rescaled_data = idf_model.transform(featurized_data)

In [51]:
rescaled_data.select('id', 'features').show()

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|(262144,[24417,49...|
|  1|(262144,[55551,88...|
|  2|(262144,[84234],[...|
+---+--------------------+



### Count Vectorizer

In [52]:
from pyspark.ml.feature import CountVectorizer

In [53]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [54]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [56]:
cv = CountVectorizer(
    inputCol='words',
    outputCol='features',
    vocabSize=3,
    minDF=2.0)

In [57]:
model = cv.fit(df)

In [58]:
results = model.transform(df)

In [59]:
results.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

